# Data Cleaning Notebook

## Notebook Summary
In this Notebook:
- All columns related to ADHD are removed
- All NaN values present in the dataset are addressed
- A train-test split is performed on the data, and a validation set is created
- transformed/cleaned data is saved as CSV files.

In [4]:

# Relevant imports
import warnings
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [5]:
nsch20 = pd.read_sas('../Data/nsch_2020_topical_SAS/nsch_2020_topical.sas7bdat')
nsch19 = pd.read_sas('../Data/nsch_2019_topical_SAS/nsch_2019_topical.sas7bdat')
nsch18 = pd.read_sas('../Data/nsch_2018_topical_SAS/nsch_2018_topical.sas7bdat')

# Joining all the data into one frame, keeping ONLY the columns that are present in all frames.
nsch = pd.concat([nsch20, nsch19, nsch18], join='inner', ignore_index=True)

## ADHD Columns

Along with our target column, 'K2Q31A' there are multiple other columns that relate to ADHD that need to be removed. These columns are closely tied to ADHD so they need to be removed so the model doesn't take them into account when making predictions. Column descriptions can be found in the [EDA notebook](https://github.com/austint1121/Undiagnosed-ADHD-Identification/blob/main/Notebooks/EDA.ipynb). We will also save our target column, and remove any rows with null values in that column

In [6]:
# Creating list of columns to be dropped
related_ADHD = [
    'K2Q31A',
    'K2Q31B',
    'K2Q31C',
    'K2Q31D',
    'K4Q23',
    'SC_K2Q10',
    'SC_K2Q11',
    'SC_K2Q12',
    'ADDTREAT',
    'SC_CSHCN',
    'SC_K2Q22',
    'SC_K2Q10',
    'K4Q22_R',
    'K6Q15',
    'SC_K2Q20',
    'K4Q36',
    'TOTNONSHCN',
    'K4Q28X04',
]

In [7]:
# Dropping rows with NAN values in target column
dropped_adhd = nsch.dropna(subset=['K2Q31A'])

# Saving Target column
target = dropped_adhd['K2Q31A']

# Creating new dataframe without columns from above
dropped_adhd = dropped_adhd.drop(columns=related_ADHD)

# Confirming expected results
dropped_adhd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102034 entries, 0 to 102739
Columns: 411 entries, FIPSST to FWC
dtypes: float64(407), object(4)
memory usage: 320.7+ MB


### Binarizing Target Values
Currently in our target column a "1" means that the child has been diagnosed with ADHD and a 2 means they haven't been. This will cause problems down the road later, so I'm going to manually replace them with 0's and 1's.

In [8]:
# Dictionary for replacement
testing = {2: 0, 1: 1}

# Preforming replacement
target = target.replace(testing)

# Visually confirming expected change
target.value_counts()


0.0    91695
1.0    10339
Name: K2Q31A, dtype: int64

### Dropping Object Columns

In [9]:
# Checking the column types
dropped_adhd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102034 entries, 0 to 102739
Columns: 411 entries, FIPSST to FWC
dtypes: float64(407), object(4)
memory usage: 320.7+ MB


In [10]:
# We have 422 float64 and 4 object types. Lets investigate those 4 objects
dropped_adhd.select_dtypes('object')

,FIPSST,STRATUM,HHID,FORMTYPE
0,b'17',b'1',b'20000003',b'T1'
1,b'29',b'2A',b'20000004',b'T3'
2,b'47',b'1',b'20000005',b'T1'
3,b'28',b'1',b'20000014',b'T3'
4,b'55',b'1',b'20000015',b'T3'
...,...,...,...,...
102735,b'33',b'1',b'18176000',b'T2'
102736,b'53',b'1',b'18176008',b'T1'
102737,b'48',b'2A',b'18176020',b'T2'
102738,b'13',b'2A',b'18176022',b'T2'


**FIPSST** - State FIPS code
**STRATUM** - Sampling Stratum
**HHID** - Unique Household ID
**FORMTYPE** - A proxy for age, kids are given a form base on age ranges (T1: 0-5, T2: 6-11, T3:12-17)

All of these columns can be dropped as they should have an effect on whether a child has ADHD, or they are a proxy for an already present variable.

In [11]:
# Dropping object columns
dropped_final = dropped_adhd.drop(columns=['FIPSST', 'STRATUM', 'HHID', 'FORMTYPE'])
# Confirming expected column count
dropped_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102034 entries, 0 to 102739
Columns: 407 entries, TOTKIDS_R to FWC
dtypes: float64(407)
memory usage: 317.6 MB


## Train Test Split
Before doing any transformations it will be necessary to perform the train test split beforehand to prevent data leakage.

In [12]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(dropped_final, target, random_state=15, stratify=target)

# Split test into a testing and final holdout/validation set
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=15, stratify=y_test)

# Printing total rows in each set
print(f'Training set is {len(X_train)} entries')
print(f'Testing set is {len(X_test)} entries')
print(f'Validation set is {len(X_val)} entries')

Training set is 76525 entries
Testing set is 19131 entries
Validation set is 6378 entries


In [13]:
# Printing the amount of kids diagnosed with ADHD in each split
print(f'There are {y_train.value_counts().values[1]} kids with ADHD in the training set')
print(f'There are {y_test.value_counts().values[1]} kids with ADHD in the testing set')
print(f'There are {y_val.value_counts().values[1]} kids with ADHD in the validation set')

There are 7754 kids with ADHD in the training set
There are 1939 kids with ADHD in the testing set
There are 646 kids with ADHD in the validation set



## Handling Missing Values
There are multiple strategies to handling missing values, normally I would love to use Sklearns experimental [Iterative Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html) however, in this survey some questions are sub questions of others, and can be left blank as a result of the answer to the parent question.
<br>
An example of this is the "ADDTREAT" column we dropped earlier. This column is left blank if "K2Q31A" (our target) is answered "No". For now, I will create an imputer but not use it yet, as it would take a large amount of time for it to run on the large dataset.

In [14]:
# Simple Imputer filling values with something that vill obviously be a nan value, in this case, 999
SI_imputer = SimpleImputer(strategy='constant', fill_value=999)

In [15]:
# Fitting the imputer to training data / transforming the training data
transformed_X_train = SI_imputer.fit_transform(X_train, y_train)

# Creating a dataframe and re-adding the column names to the data
transformed_X_train = pd.DataFrame(transformed_X_train, columns=X_train.columns)

# Visually confirming transformation
transformed_X_train.head()

,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,K2Q35A_1_YEARS,MOMAGE,K6Q41R_STILL,K6Q42R_NEVER,K6Q43R_NEVER,...,A1_GRADE_IF,BMICLASS,HHCOUNT_IF,FPL_I1,FPL_I2,FPL_I3,FPL_I4,FPL_I5,FPL_I6,FWC
0,2.0,1.0,1.0,7.0,1.0,999.0,35.0,999.0,999.0,999.0,...,0.0,999.0,0.0,400.0,400.0,400.0,400.0,400.0,400.0,946.327377
1,2.0,1.0,1.0,9.0,1.0,999.0,28.0,999.0,999.0,999.0,...,0.0,999.0,0.0,400.0,400.0,400.0,400.0,400.0,400.0,2283.583410
2,3.0,1.0,1.0,6.0,2.0,999.0,31.0,999.0,999.0,999.0,...,0.0,999.0,0.0,176.0,220.0,153.0,175.0,400.0,400.0,258.317200
3,1.0,1.0,1.0,6.0,2.0,999.0,34.0,999.0,999.0,999.0,...,0.0,999.0,0.0,400.0,400.0,400.0,400.0,400.0,400.0,2549.241201
4,1.0,2.0,3.0,3.0,2.0,999.0,30.0,2.0,1.0,1.0,...,0.0,999.0,0.0,104.0,104.0,104.0,104.0,104.0,104.0,93.141525


In [16]:
# Transforming test and validation set
transformed_X_test = SI_imputer.transform(X_test)

# Creating a dataframe and re-adding the column names to the data
transformed_X_test= pd.DataFrame(transformed_X_test, columns=X_test.columns)

# Transforming Validation set
transformed_X_val = SI_imputer.transform(X_val)

# Creating a dataframe and re-adding the column names to the data
transformed_X_val= pd.DataFrame(transformed_X_val, columns=X_val.columns)


## Conclusion

In [14]:
# Saving the transformed dataframes

# Training Data
transformed_X_train.to_csv('../Data/train/X_train_combo.csv')
y_train.to_csv('../Data/train/y_train_combo.csv')

# Testing Data
transformed_X_test.to_csv('../Data/test/X_test_combo.csv')
y_test.to_csv('../Data/test/y_test_combo.csv')

# Validation Data
transformed_X_val.to_csv('../Data/val/X_val_combo.csv')
y_val.to_csv('../Data/val/y_val_combo.csv')

From this point forward, any more cleaning would be model specific. I could also clean the data more to improve the overall quality, but I think this will be a good "baseline" starting point for the modeling process. The next step is to begin the [modeling proccess](Modeling/First_Simple_Models.ipynb)